# Your Title Here

**Name(s)**: (your name(s) here)

**Website Link**: (your website link)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
pd.options.plotting.backend = 'plotly'


from dsc80_utils import * # Feel free to uncomment and use this.

In [2]:
import plotly.io as pio
pio.renderers.default = 'jupyterlab'

In [3]:
from datetime import time

## Step 1: Introduction

1. We picked outage dataset because it is rich in information as well as something we have a bit of knowledge in. Also because it is interesting to see the result and analyze what causes power outages
2. plotly vis
3. hypotheses
4. prediction?

We will be answering the following question:  
what characteristic of power outages causes the most economic loss?

## Step 2: Data Cleaning and Exploratory Data Analysis

In [4]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [5]:
original = pd.read_csv('data/power_outage.csv')

In [6]:
original

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,"Friday, July 01, 2011",5:00:00 PM,"Sunday, July 03, 2011",8:00:00 PM,severe weather,NaN,NaN,3060.0,NaN,70000.0,11.60,9.18,6.81,9.28,2.33e+06,2.11e+06,2.11e+06,6.56e+06,35.55,32.23,32.20,2308736,276286,10673,2595696,88.94,10.64,0.41,51268,47586,1.08,1.6,4802,274182,1.75,2.2,5348119,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.59,8.41,5.48
1,2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,"Sunday, May 11, 2014",6:38:00 PM,"Sunday, May 11, 2014",6:39:00 PM,intentional attack,vandalism,NaN,1.0,NaN,NaN,12.12,9.71,6.49,9.28,1.59e+06,1.81e+06,1.89e+06,5.28e+06,30.03,34.21,35.73,2345860,284978,9898,2640737,88.83,10.79,0.37,53499,49091,1.09,1.9,5226,291955,1.79,2.2,5457125,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.59,8.41,5.48
2,3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,"Tuesday, October 26, 2010",8:00:00 PM,"Thursday, October 28, 2010",10:00:00 PM,severe weather,heavy wind,NaN,3000.0,NaN,70000.0,10.87,8.19,6.07,8.15,1.47e+06,1.80e+06,1.95e+06,5.22e+06,28.10,34.50,37.37,2300291,276463,10150,2586905,88.92,10.69,0.39,50447,47287,1.07,2.7,4571,267895,1.71,2.1,5310903,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.59,8.41,5.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,1532,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,"Saturday, August 29, 2009",10:54:00 PM,"Saturday, August 29, 2009",11:53:00 PM,islanding,NaN,NaN,59.0,84.0,NaN,9.25,7.47,5.53,7.67,3.38e+05,3.71e+05,2.15e+05,9.24e+05,36.56,40.12,23.31,367206,65971,3052,436229,84.18,15.12,0.70,45230,46680,0.97,0.0,606,36504,1.66,0.3,807067,56.65,26.73,2038.3,1905.4,4.7,0.30,0.15,98.31,1.69,1.69
1532,1533,2009,8.0,South Dakota,SD,MRO,West North Central,0.5,warm,"Saturday, August 29, 2009",11:00:00 AM,"Saturday, August 29, 2009",2:01:00 PM,islanding,NaN,NaN,181.0,373.0,NaN,9.25,7.47,5.53,7.67,3.38e+05,3.71e+05,2.15e+05,9.24e+05,36.56,40.12,23.31,367206,65971,3052,436229,84.18,15.12,0.70,45230,46680,0.97,0.0,606,36504,1.66,0.3,807067,56.65,26.73,2038.3,1905.4,4.7,0.30,0.15,98.31,1.69,1.69
1533,1534,2000,NaN,Alaska,AK,ASCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,equipment failure,failure,NaN,NaN,35.0,14273.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230534,38074,854,273530,84.28,13.92,0.31,57401,44745,1.28,-2.2,724,36046,2.01,0.2,627963,66.02,21.56,1802.6,1276.0,0.4,0.05,0.02,85.76,14.24,2.90


In [7]:
original['OUTAGE.DURATION'].isna().sum()

np.int64(58)

In [8]:
original[['OUTAGE.START.DATE','OUTAGE.START.TIME','OUTAGE.RESTORATION.DATE','OUTAGE.RESTORATION.TIME']].isna().sum()

OUTAGE.START.DATE           9
OUTAGE.START.TIME           9
OUTAGE.RESTORATION.DATE    58
OUTAGE.RESTORATION.TIME    58
dtype: int64

In [9]:
original[original['OUTAGE.DURATION']==0]

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
12,13,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,"Friday, July 08, 2011",10:00:00 AM,"Friday, July 08, 2011",10:00:00 AM,intentional attack,vandalism,NaN,0.0,0.0,0.0,11.60,9.18,6.81,9.28,2.33e+06,2.11e+06,2.11e+06,6.56e+06,35.55,32.23,32.20,2308736,276286,10673,2595696,88.94,10.64,0.41,51268,47586,1.08,1.6,4802,274182,1.75,2.2,5348119,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.59,8.41,5.48
16,17,2012,8.0,Tennessee,TN,SERC,Central,0.3,normal,"Monday, August 13, 2012",1:15:00 PM,"Monday, August 13, 2012",1:15:00 PM,intentional attack,vandalism,NaN,0.0,NaN,NaN,10.00,10.21,7.75,9.47,4.40e+06,2.83e+06,2.56e+06,9.79e+06,44.95,28.94,26.11,2721099,466513,1957,3189570,85.31,14.63,0.06,41469,48156,0.86,2.5,1819,267689,0.68,0.4,6455469,66.39,12.02,1450.3,1076.2,55.6,7.05,1.72,97.84,2.16,2.16
26,27,2011,6.0,Tennessee,TN,SERC,Central,-0.3,normal,"Wednesday, June 22, 2011",9:46:00 AM,"Wednesday, June 22, 2011",9:46:00 AM,severe weather,NaN,NaN,0.0,NaN,106300.0,9.98,10.29,7.71,9.47,3.72e+06,2.64e+06,2.31e+06,8.66e+06,42.89,30.43,26.68,2708113,463992,2017,3174123,85.32,14.62,0.06,40465,47586,0.85,2.0,1798,258910,0.69,0.4,6398408,66.39,12.02,1450.3,1076.2,55.6,7.05,1.72,97.84,2.16,2.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,1481,2011,5.0,New Hampshire,NH,NPCC,Northeast,-0.4,normal,"Monday, May 09, 2011",1:11:00 PM,"Monday, May 09, 2011",1:11:00 PM,intentional attack,vandalism,NaN,0.0,0.0,0.0,16.90,14.32,12.47,14.96,3.01e+05,3.30e+05,1.49e+05,7.80e+05,38.57,42.34,19.09,599532,104610,3491,707633,84.72,14.78,0.49,47697,47586,1.00,0.9,1328,62870,2.11,0.5,1318344,60.30,12.97,1232.6,1364.9,62.9,7.19,1.40,95.76,4.25,3.51
1521,1522,2011,12.0,Idaho,ID,WECC,Northwest,-0.9,cold,"Wednesday, December 21, 2011",10:30:00 AM,"Wednesday, December 21, 2011",10:30:00 AM,intentional attack,vandalism,NaN,0.0,0.0,0.0,7.43,5.76,4.28,6.12,9.40e+05,5.32e+05,5.60e+05,2.03e+06,46.26,26.18,27.56,667243,101424,26258,794925,83.94,12.76,3.30,34481,47586,0.72,-0.7,1078,54611,1.97,0.4,1584134,70.58,20.06,2216.8,2004.7,5.6,0.60,0.19,98.89,1.11,1.11
1526,1527,2016,3.0,Idaho,ID,WECC,Northwest,1.6,warm,"Tuesday, March 08, 2016",12:00:00 AM,"Tuesday, March 08, 2016",12:00:00 AM,intentional attack,sabotage,NaN,0.0,0.0,0.0,9.80,7.66,5.98,8.05,6.78e+05,4.99e+05,4.80e+05,1.66e+06,40.93,30.13,28.94,714365,107551,27847,849763,84.07,12.66,3.28,36256,50660,0.72,2.7,960,60911,1.58,0.4,1680026,70.58,20.06,2216.8,2004.7,5.6,0.60,0.19,98.89,1.11,1.11


Select the following columns for Duration analysis

In [10]:
outage = original[['YEAR','MONTH','U.S._STATE','NERC.REGION', 'CLIMATE.REGION', 
                   'ANOMALY.LEVEL', 'CLIMATE.CATEGORY','OUTAGE.START.DATE', 'OUTAGE.START.TIME', 
                   'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 
                   'CAUSE.CATEGORY.DETAIL','HURRICANE.NAMES', 'OUTAGE.DURATION', 
                   'DEMAND.LOSS.MW','CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 
                   'IND.PRICE','TOTAL.PRICE','POPULATION', 'AREAPCT_URBAN', 'PCT_LAND']]
outage

,YEAR,MONTH,U.S._STATE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,POPULATION,AREAPCT_URBAN,PCT_LAND
0,2011,7.0,Minnesota,MRO,East North Central,-0.3,normal,"Friday, July 01, 2011",5:00:00 PM,"Sunday, July 03, 2011",8:00:00 PM,severe weather,NaN,NaN,3060.0,NaN,70000.0,11.60,9.18,6.81,9.28,5348119,2.14,91.59
1,2014,5.0,Minnesota,MRO,East North Central,-0.1,normal,"Sunday, May 11, 2014",6:38:00 PM,"Sunday, May 11, 2014",6:39:00 PM,intentional attack,vandalism,NaN,1.0,NaN,NaN,12.12,9.71,6.49,9.28,5457125,2.14,91.59
2,2010,10.0,Minnesota,MRO,East North Central,-1.5,cold,"Tuesday, October 26, 2010",8:00:00 PM,"Thursday, October 28, 2010",10:00:00 PM,severe weather,heavy wind,NaN,3000.0,NaN,70000.0,10.87,8.19,6.07,8.15,5310903,2.14,91.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,2009,8.0,South Dakota,RFC,West North Central,0.5,warm,"Saturday, August 29, 2009",10:54:00 PM,"Saturday, August 29, 2009",11:53:00 PM,islanding,NaN,NaN,59.0,84.0,NaN,9.25,7.47,5.53,7.67,807067,0.30,98.31
1532,2009,8.0,South Dakota,MRO,West North Central,0.5,warm,"Saturday, August 29, 2009",11:00:00 AM,"Saturday, August 29, 2009",2:01:00 PM,islanding,NaN,NaN,181.0,373.0,NaN,9.25,7.47,5.53,7.67,807067,0.30,98.31
1533,2000,NaN,Alaska,ASCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,equipment failure,failure,NaN,NaN,35.0,14273.0,NaN,NaN,NaN,NaN,627963,0.05,85.76


Change the 0s in 'DEMAND.LOSS.MW' and 'CUSTOMERS.AFFECTED' to np.nan because it's unlikely for them to be 0

In [11]:
outage['DEMAND.LOSS.MW']=outage['DEMAND.LOSS.MW'].apply(lambda x: np.nan if x==0 else x)
outage['CUSTOMERS.AFFECTED']=outage['CUSTOMERS.AFFECTED'].apply(lambda x: np.nan if x==0 else x)

Change the date and time columns to pd.datetime objects for easier operations

In [12]:
outage[['OUTAGE.START.DATE', 'OUTAGE.RESTORATION.DATE']]=outage[['OUTAGE.START.DATE','OUTAGE.RESTORATION.DATE']].apply(pd.to_datetime)
outage[['OUTAGE.START.TIME','OUTAGE.RESTORATION.TIME']]=outage[['OUTAGE.START.TIME','OUTAGE.RESTORATION.TIME']].apply(lambda x: pd.to_datetime(x).dt.time)

/var/folders/s9/b35x2lvd0zl66546s25wwsjc0000gn/T/ipykernel_89443/4267996805.py:2: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/var/folders/s9/b35x2lvd0zl66546s25wwsjc0000gn/T/ipykernel_89443/4267996805.py:2: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



Create a column 'AWAKE.OR.SLEEP' to determine whether people are usually awake when the outage happen

In [13]:
outage['AWAKE.OR.SLEEP']=outage['OUTAGE.START.TIME'].apply(lambda x: x if pd.isna(x) else('Awaken' if x>time(6, 0) and x<time(22) else 'Sleeping'))

In [14]:
outage

,YEAR,MONTH,U.S._STATE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,POPULATION,AREAPCT_URBAN,PCT_LAND,AWAKE.OR.SLEEP
0,2011,7.0,Minnesota,MRO,East North Central,-0.3,normal,2011-07-01,17:00:00,2011-07-03,20:00:00,severe weather,NaN,NaN,3060.0,NaN,70000.0,11.60,9.18,6.81,9.28,5348119,2.14,91.59,Awaken
1,2014,5.0,Minnesota,MRO,East North Central,-0.1,normal,2014-05-11,18:38:00,2014-05-11,18:39:00,intentional attack,vandalism,NaN,1.0,NaN,NaN,12.12,9.71,6.49,9.28,5457125,2.14,91.59,Awaken
2,2010,10.0,Minnesota,MRO,East North Central,-1.5,cold,2010-10-26,20:00:00,2010-10-28,22:00:00,severe weather,heavy wind,NaN,3000.0,NaN,70000.0,10.87,8.19,6.07,8.15,5310903,2.14,91.59,Awaken
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,2009,8.0,South Dakota,RFC,West North Central,0.5,warm,2009-08-29,22:54:00,2009-08-29,23:53:00,islanding,NaN,NaN,59.0,84.0,NaN,9.25,7.47,5.53,7.67,807067,0.30,98.31,Sleeping
1532,2009,8.0,South Dakota,MRO,West North Central,0.5,warm,2009-08-29,11:00:00,2009-08-29,14:01:00,islanding,NaN,NaN,181.0,373.0,NaN,9.25,7.47,5.53,7.67,807067,0.30,98.31,Awaken
1533,2000,NaN,Alaska,ASCC,NaN,NaN,NaN,NaT,NaT,NaT,NaT,equipment failure,failure,NaN,NaN,35.0,14273.0,NaN,NaN,NaN,NaN,627963,0.05,85.76,NaT


UNIVARIATE ANALYSIS

In [15]:
fig1 = px.histogram(outage, x='POPULATION')
fig2 = px.histogram(outage, x='OUTAGE.DURATION')
climate_ct_group = outage.groupby('CLIMATE.CATEGORY')['OUTAGE.DURATION'].count()
fig3 = px.bar(climate_ct_group,labels={'CLIMATE.CATEGORY':'Climate','value':'Count'})

fig1.show()
fig2.show()
fig3.show()

BIVARIATE ANALYSIS

In [16]:
fig1 = px.scatter(outage, x='POPULATION', y='OUTAGE.DURATION')
fig2 = px.scatter(outage, x='ANOMALY.LEVEL', y='OUTAGE.DURATION',title='Scatter Plot: ANOMALYLLYRL vs. OUTAGE DURATION',
                  labels={'ANOMALY.LEVEL': 'Anomaly Level', 'OUTAGE.DURATION': 'Outage Duration (hours)'})
fig3 = px.box(outage, x='CAUSE.CATEGORY', y='OUTAGE.DURATION',
              title='Box Plot: OUTAGE DURATION by CAUSE CATEGORY',
              labels={'CAUSE.CATEGORY': 'Cause Category', 'OUTAGE.DURATION': 'Outage Duration (hours)'})
fig4 = px.histogram(outage, x='ANOMALY.LEVEL', color='CLIMATE.CATEGORY', barmode='overlay',
                    title='Conditional Distribution: ANOMALYLLYRL by CLIMATE CATEGORY',
                    labels={'ANOMALY.LEVEL': 'Anomaly Level', 'count': 'Frequency'})
outage['OUTAGE.DURATION']=outage['OUTAGE.DURATION'].astype('float')
climate_avg_duration = outage.groupby('CLIMATE.CATEGORY')['OUTAGE.DURATION'].mean().reset_index()
fig5 = px.bar(climate_avg_duration, x='CLIMATE.CATEGORY', y='OUTAGE.DURATION',
               title='Average Outage Duration by CLIMATE CATEGORY',
               
               labels={'CLIMATE.CATEGORY': 'Climate Category', 'OUTAGE.DURATION': 'Average Outage Duration (hours)'})




# Heatmap: Outage Count by CAUSE CATEGORY and CLIMATE CATEGORY
pivot_table = pd.pivot_table(outage, values='OUTAGE.DURATION', index='CAUSE.CATEGORY', columns='CLIMATE.CATEGORY', aggfunc='count')
fig6 = px.imshow(pivot_table, labels=dict(x="Climate Category", y="Cause Category", color="Outage Count"),
                 title='Outage Count by CAUSE CATEGORY and CLIMATE CATEGORY')


# Bar plot: Maximum Anomaly Level by STATE
outage['ANOMALY.LEVEL']=outage['ANOMALY.LEVEL'].astype('float')
state_max_anomaly = outage.groupby('U.S._STATE')['ANOMALY.LEVEL'].mean().reset_index()
fig7 = px.bar(state_max_anomaly, x='U.S._STATE', y='ANOMALY.LEVEL',
              title='Maximum Anomaly Level by STATE',
              labels={'U.S._STATE': 'State', 'ANOMALY.LEVEL': 'Maximum Anomaly Level'})
fig8 = px.scatter(outage, x='ANOMALY.LEVEL', y='OUTAGE.DURATION')
fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()
fig6.show()
fig7.show()
fig8.show()


Missingness


In [17]:
outage['CAUSE CATEGORY DI_MISSING'] = outage['CAUSE.CATEGORY.DETAIL'].isna().astype(int)
observed = outage.groupby('CLIMATE.CATEGORY')['CAUSE CATEGORY DI_MISSING'].mean()
observed

CLIMATE.CATEGORY
cold      0.35
normal    0.27
warm      0.35
Name: CAUSE CATEGORY DI_MISSING, dtype: float64

In [18]:
def permutation_test(df, group_col, missing_col, n_permutations=1000):
    observed_diff = df.groupby(group_col)[missing_col].mean().diff().abs().sum()
    perm_diffs = []
    for _ in range(n_permutations):
        df_perm = df.copy()
        df_perm[missing_col] = np.random.permutation(df_perm[missing_col])
        perm_diff = df_perm.groupby(group_col)[missing_col].mean().diff().abs().sum()
        perm_diffs.append(perm_diff)
    p_value = (np.array(perm_diffs) >= observed_diff).mean()
    return p_value


In [19]:
p_value = permutation_test(outage, 'CLIMATE.CATEGORY', 'CAUSE CATEGORY DI_MISSING')
p_value

np.float64(0.0)

After doing the permutation test on climate category and cause category missingness, the pval came out to be 0.001  
Since pval<0.05, this means missingness is very likely to be dependent on climate category. Additonally, this means cause missingness is Missing at Random (MAR)

## Step 3: Assessment of Missingness

* NMAR Analysis

Among all columns with missing values, 'DEMAND.LOSS.MW' and 'CUSTOMERS.AFFECTED' has a high chance of being NMAR. Because these values could be missing due to the data collection method. As these values could bring direct and crucial negative effects to the power companies, they might not report the data to the institute collecting the data. 
With addition information about the companies such as their name, their size, or their annual revenue, it is possible to determine whether the missing values are dependent on the companies or their performances and thus making them MAR. 

* Missingness Dependency

In [20]:
def permutation_test(df, group_col, missing_col, n_permutations=1000):
    observed_diff = df.groupby(group_col)[missing_col].mean().diff().abs().sum()
    perm_diffs = []
    for _ in range(n_permutations):
        df_perm = df.copy()
        df_perm[missing_col] = np.random.permutation(df_perm[missing_col])
        perm_diff = df_perm.groupby(group_col)[missing_col].mean().diff().abs().sum()
        perm_diffs.append(perm_diff)
    p_value = (np.array(perm_diffs) >= observed_diff).mean()
    return p_value

1. Test 'OUTAGE.RESTORATION.DATE' with 'NERC.REGION'

2. Test 'OUTAGE.RESTORATION.DATE' with 'MONTH'

## Step 4: Hypothesis Testing

We will be testing whether Duration is longer when it begins at sleep time. The columns that are use for testing are 'OUTAGE.DURATION' and 'AWAKE.OR.SLEEP'.

- Null Hypothesis: The average duration when it begins during usual sleep time is the same as the duration that begins during usual awaken time. 
- Alternative Hypothesis: The average duration when it begins during usual sleep time is longer than the duration that begins during usual awaken time.
- Test Statistic: Difference in mean (Mean duration during normal sleep time - Mean duration during normal awaken time).

In [21]:
df=outage.dropna(subset='OUTAGE.DURATION')

level=0.01
obs=df[df['AWAKE.OR.SLEEP']=='Sleeping']['OUTAGE.DURATION'].mean()-\
    df[df['AWAKE.OR.SLEEP']=='Awaken']['OUTAGE.DURATION'].mean()
print(f'observed stat: {obs}')
samp_diff=[]
n=10000
for i in range(n):
    df['AWAKE.OR.SLEEP']=np.random.permutation(df['AWAKE.OR.SLEEP'])
    diff=df[df['AWAKE.OR.SLEEP']=='Sleeping']['OUTAGE.DURATION'].mean()-\
        df[df['AWAKE.OR.SLEEP']=='Awaken']['OUTAGE.DURATION'].mean()
    samp_diff.append(diff)

p=(samp_diff>obs).sum()/n
print(f'p-value: {p}')

if p<level:
    print('Reject null.')
    print('The average duration when it begins during usual sleep time is longer than '
    'the duration that begins during usual awaken time.')
else:
    print('Fail to reject null.')
    print('The average duration when it begins during usual sleep time is NOT '
    'longer than the duration that begins during usual awaken time.')

observed stat: 1127.5740267713254
p-value: 0.0034
Reject null.
The average duration when it begins during usual sleep time is longer than the duration that begins during usual awaken time.


In [22]:
# Create histogram for samp
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=samp_diff,
        nbinsx=30,
        name="Simulated Test Statistics",
        marker=dict(color="skyblue"),
        opacity=0.7
    )
)

# Vertical line for obs
fig.add_trace(
    go.Scatter(
        x=[obs, obs],
        y=[0, max(np.histogram(samp_diff, bins=30)[0])],
        mode="lines",
        name=f"Observed Statistic = {obs:.2f}",
        line=dict(color="red", dash="dash")
    )
)

# Customize layout
fig.update_layout(
    title="Hypothesis Test Simulation",
    xaxis_title="Test Statistic",
    yaxis_title="Frequency",
    legend=dict(font=dict(size=12)),
    template="plotly_white"
)

# Show the plot
fig.show()


## Step 5: Framing a Prediction Problem

We are building a model to predict the outage duration, so it's a regression problem. We will be using to Polynomial Regression to try to fit our data because we are not sure if the relationship between outage duration and other variables are linear. 

The metric we're using to evaluate our model is R Square because we are not planning to use too many columns so the value of R Square won't get inflatted as the increase of polynomial degree (also it's easily accessable).

We will make out prediction base on some of the following values: year, month, state, NERC regin, climate regin, anomaly level, climate category, beginning time of the outage, cause category, hurricane, number of customers affected, average price of electricity, percentage of land area, and whether people are awake or not whent the outage begins. 

In [23]:
# TODO

## Step 6: Baseline Model

We will be using the following 5 parameter to train our model: month, population in the state, average price of electricity, percentage of urban land area, and whether people are awake or not. 

(justify why dropping rows)

In [ ]:
# Import necessary libraries
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline


for_train=outage.dropna(subset=['OUTAGE.DURATION'])
X=for_train[['CUSTOMERS.AFFECTED', 'POPULATION', 'CAUSE.CATEGORY', 'CLIMATE.REGION']]
y=for_train['OUTAGE.DURATION']

num_trans=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])
cat_trans=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_trans, ['CUSTOMERS.AFFECTED', 'POPULATION']),  # Scale numerical features
        ('cat', cat_trans, ['CAUSE.CATEGORY', 'CLIMATE.REGION'])       # One-hot encode categorical features
    ]
)

# Transform the features
X_preprocessed = preprocessor.fit_transform(X)

# Step 4: Apply Polynomial Features Transformation
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_preprocessed)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

# Step 6: Train the Polynomial Regression model (Linear Regression on transformed data)
model = LinearRegression()
model.fit(X_train, y_train)

# Step 7: Make predictions and evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

r2


NameError: name 'Pipeline' is not defined

## Step 7: Final Model

- hyperparameter
- add new features
- probably change the model
- do anything to make it better

In [ ]:
# TODO

## Step 8: Fairness Analysis

In [ ]:
# TODO

References:
- https://docs.lib.purdue.edu/cgi/viewcontent.cgi?params=/context/civeng/article/1035/&path_info=10.1016_j.dib.2018.06.067.pdf